<a href="https://colab.research.google.com/github/E-CG/AI4ENG/blob/master/02%20-%20Preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **📦 Librerias y paquetes para la ejecución del notebook.**

In [ ]:
! pip install py7zr

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle
# Aquí debes cambiar la dirección donde tengas tus credenciales de Kaggle
! cp /content/drive/MyDrive/Modelos_I/credentials_kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download favorita-grocery-sales-forecasting
! unzip favorita-grocery-sales-forecasting.zip

In [9]:
# Librerias uso básico
import numpy as np
import pandas as pd
import math as m
import time
import py7zr
import os
from subprocess import check_output

# Librerias preprocesado
from mlxtend.preprocessing import minmax_scaling

# Librerias para gráficar
import seaborn as sns
import matplotlib.pyplot as plt

# Funciones de sklearn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression,SGDRegressor,ElasticNet,Ridge
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error

# **🗂️Leyendo y extrayendo los archivos .csv**

In [7]:
# Ruta al archivo 7z en Google Drive
sevenzip_file_path = '/content'

# Directorio de destino para la extracción
extracted_dir = '/content/extracted_data/'

# Crear el directorio de destino si no existe
os.makedirs(extracted_dir, exist_ok=True)

# Listar los archivos .7z en el directorio de entrada
files_to_extract = [file for file in os.listdir(sevenzip_file_path) if file.endswith('.7z')]

# Iterar a través de los archivos y descomprimirlos
for file_to_extract in files_to_extract:
    with py7zr.SevenZipFile(os.path.join(sevenzip_file_path, file_to_extract), mode='r') as z:
        z.extractall(path=extracted_dir)

In [32]:
stores = pd.read_csv('/content/extracted_data/stores.csv')
items = pd.read_csv('/content/extracted_data/items.csv')
holidays_e = pd.read_csv('/content/extracted_data/holidays_events.csv', parse_dates=["date"])
transactions = pd.read_csv('/content/extracted_data/transactions.csv', parse_dates=["date"])
oil = pd.read_csv('/content/extracted_data/oil.csv')

# Cargar el archivo de entrenamiento en chunks
chunked_dfs = pd.read_csv("/content/extracted_data/train.csv",
                          chunksize=20000,
                          usecols=[1, 2, 3, 4, 5],
                          parse_dates=['date'],
                          low_memory=False)

print('Archivos cargados 🗞️✅')

Archivos cargados 🗞️✅


# **❌¿Qué archivos tienen datos nulos?**

🖊️ Se toma cada columna del dataframe en cuestion, se suman la cantidad de registros NaN y se convierte en un **porcentaje**. A cada columna se le saca tal porcentaje.

In [ ]:
oil_nan = (oil.isnull().sum() / oil.shape[0]) * 100
oil_nan

date          0.000000
dcoilwtico    3.530378
dtype: float64

Hay un 3.5% de datos faltantes en el archivo (oil.csv)

In [ ]:
store_nan = (stores.isnull().sum() / stores.shape[0]) * 100
store_nan

store_nbr    0.0
city         0.0
state        0.0
type         0.0
cluster      0.0
dtype: float64

No hay datos faltantes en (stores.csv)

In [ ]:
item_nan = (items.isnull().sum() / items.shape[0]) * 100
item_nan

item_nbr      0.0
family        0.0
class         0.0
perishable    0.0
dtype: float64

No hay datos faltantes en (items.csv)

In [ ]:
holiday_nan = (holidays_e.isnull().sum() / holidays_e.shape[0]) * 100
holiday_nan

date           0.0
type           0.0
locale         0.0
locale_name    0.0
description    0.0
transferred    0.0
dtype: float64

No hay datos nulo en el archivo (holidays_events.csv)

In [ ]:
tran_nan = (transactions.isnull().sum() / transactions.shape[0]) * 100
tran_nan

date            0.0
store_nbr       0.0
transactions    0.0
dtype: float64

No hay datos faltantes en (transactions.csv)

# **🦾Primeras predicciones**

Tomar 1 tienda por estado en las fechas mayores a 2016 y predecir las ventas unitarias de todos los productos.

In [12]:
# Agrupar por estado y obtener el primer store_nbr en cada estado
tienda_por_estado = stores.groupby('state')['store_nbr'].first()
tienda_por_estado

state
Azuay                             37
Bolivar                           19
Chimborazo                        14
Cotopaxi                          12
El Oro                            40
Esmeraldas                        43
Guayas                            24
Imbabura                          15
Loja                              38
Los Rios                          31
Manabi                            52
Pastaza                           22
Pichincha                          1
Santa Elena                       25
Santo Domingo de los Tsachilas     5
Tungurahua                        23
Name: store_nbr, dtype: int64

In [33]:
# Inicializar una lista para almacenar los DataFrames filtrados
datos_filtrados = []

# Iterar a través de los chunks
for chunk in chunked_dfs:
    # Crear una máscara booleana para las tiendas seleccionadas
    mask_tiendas = chunk['store_nbr'].isin(stores['store_nbr'])
    # Crear una máscara booleana para las fechas entre 2016-01-01 a 2016-12-31
    mask = (chunk['date'] >= '2016-01-01') & (chunk['date'] <= '2016-12-31') & chunk['store_nbr'].isin(tienda_por_estado)

    # Aplicar la máscara y agregar las filas filtradas a la lista
    trozo_filtrado = chunk[mask]
    datos_filtrados.append(trozo_filtrado)

# Concatenar los DataFrames filtrados en uno solo
train_store_state = pd.concat(datos_filtrados, ignore_index=True)

In [ ]:
train_nan = (train_store_state.isnull().sum() / train_store_state.shape[0]) * 100
train_nan

date           0.0
store_nbr      0.0
item_nbr       0.0
unit_sales     0.0
onpromotion    0.0
dtype: float64

No hay datos nulos en el dataframe de entrenamiento

Limpiando datos faltantes del archivo oil.csv

In [14]:
oil['dcoilwtico'].fillna(method='bfill', inplace=True)
oil

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


Empezar a utilizar regresion linear

In [34]:
# onpromotion es una columna del dataframe 'train' se puede cambiar los trues por 1 y los false por 0
train_store_state['onpromotion'] = train_store_state['onpromotion'].replace(True,1)
train_store_state['onpromotion'] = train_store_state['onpromotion'].replace(False,0)

In [35]:
train_store_state['date'] = pd.to_datetime(train_store_state['date'])
holidays_e['date'] = pd.to_datetime(holidays_e['date'])
oil['date'] = pd.to_datetime(oil['date'])

In [79]:
stores.drop(columns=['state', 'type', 'cluster'], inplace= True)

## **📄Juntando los demás archivos en train.csv**

In [1]:
class merge_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("Juntando los archivos")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        pre_data = pd.merge(pd.merge(pd.merge(pd.merge(X, stores, on='store_nbr', how='left'),
                                     oil, on='date', how='left'), holidays_e, on='date', how='left'), items, on='item_nbr', how='left')
        pre_data = pre_data.sort_values('date')

        pre_data.dcoilwtico = pre_data.dcoilwtico.fillna(method='bfill', axis=0).fillna(0)
        pre_data.type = pre_data.type.fillna('Normal')
        pre_data.locale = pre_data.locale.fillna('Normal')

        encoder = OneHotEncoder()
        categorical_cols = ['store_nbr', 'item_nbr', 'city', 'type', 'locale', 'family']

        encoder.fit(pre_data[categorical_cols])
        encoded_cols = encoder.transform(pre_data[categorical_cols]).toarray()
        encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(categorical_cols))

        pre_data = pd.concat([pre_data.drop(columns=categorical_cols), encoded_df], axis=1)
        data = pre_data.reset_index().drop(columns='index')

        return data

NameError: ignored

## **🔎Utilizando algunos modelos para hacer las primeras predicciones**

In [86]:
class split_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("Generando conjuntos de entrenamiento y de prueba")

    def fit(self, X, y=None):
        return self

    def transform(self, X):

        y = X.pop('unit_sales')
        X = X

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

        X_train = X_train.set_index('date')
        X_test = X_test.set_index('date')

        return X_train, X_test, y_train, y_test

In [87]:
pipeline = Pipeline([
    ('merge_data', merge_data()),
    ('split_data', split_data())
])

Juntando los archivos
Generando conjuntos de entrenamiento y de prueba


In [88]:
train_sample = train_store_state.sample(n = 100000)

productos_sample = train_sample['item_nbr'].unique()

productos_sample = productos_sample[:3]

In [ ]:
X_train, X_test, y_train, y_test = pipeline.fit_transform(train_sample)

In [27]:
# Usando modelo de regresión lineal
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

# Realiza predicciones en el conjunto de prueba
predictions_lr = model_lr.predict(X_test)

# Evalúa el rendimiento del modelo de regresión lineal
mse_lr = mean_squared_error(y_test, predictions_lr)
print(f'Mean Squared Error (Linear Regression): {mse_lr}')

ValueError: ignored

In [ ]:
# Usando modelo ElasticNet
model_en = ElasticNet(alpha=0.5, l1_ratio=0.5)
model_en.fit(X_train, y_train)

# Realiza predicciones en el conjunto de prueba
predictions_en = model_en.predict(X_test)

# Evalúa el rendimiento del modelo ElasticNet
mse_en = mean_squared_error(y_test, predictions_en)
print(f'Mean Squared Error (ElasticNet): {mse_en}')